In [1]:
import requests
from bs4 import BeautifulSoup 
import lxml
import csv
from datetime import datetime


#### Taking day date from user

In [2]:
# Accept input from user
date_string = input("Enter a date in the format mm/dd/yyyy: ")
print("You entered: ", date_string)

from datetime import datetime

# Convert date string to datetime object
date_object = datetime.strptime(date_string, '%m/%d/%Y').date()
print("After changing it to data: ", date_object)

# Convert datetime object back to string with new format
new_date_string = datetime.strftime(date_object, '%m-%d-%Y')
print("After converted back to string on the format %m-%d-%Y: ", new_date_string)


Enter a date in the format mm/dd/yyyy: 3/3/2023
You entered:  3/3/2023
After changing it to data:  2023-03-03
After converted back to string on the format %m-%d-%Y:  03-03-2023


In [25]:
# Storing file name and file path in global variables to be used in multiple cells
file_name = f"Match_Data_Scraped_{new_date_string}.csv"
file_path = "C:/Users/yusuf/Data Science/Projects/Web Scraping/Yallakor.com"
path = file_path+"/"+file_name

In [3]:
# Requesting the webpage and adding the date to it
page = requests.get(f"https://www.yallakora.com/match-center?date={date_string}")
website_URL = "https://www.yallakora.com"

# Checking the response of the getting the page, if respose is 200; this means we got it fine
page

<Response [200]>

### Building function that extract match details through segment of the webpage and store it in csv file

In [39]:
def get_match_info(championships):
        # Looping through the championships
        #for championship in championships:
        
        match_details = []
        
        # Looping thorou championships
        for championship in championships:
            
            #championship_title = championships[1].find("h2").text.strip()
            championship_title = championship.find("a",{"class":"tourTitle"}).find_next("h2").text.strip()
            team_a = championship.find("div",{"class":"allData"}
                                      ).find_next("div", {"class":"teamsData"}
                                                 ).find_next("div", {"class":"teams teamA"}
                                                            ).find_next("p").text
            
            team_b = championship.find("div",{"class":"allData"}
                                      ).find_next("div", {"class":"teamsData"}
                                                 ).find_next("div", {"class":"teams teamB"}
                                                            ).find_next("p").text
            
            time = championship.find("div",{"class":"allData"}
                                      ).find_next("div", {"class":"teamsData"}
                                                 ).find_next("div", {"class":"MResult"}
                                                            ).find_next("span"
                                                                       ).find_next("span"
                                                                                  ).find_next("span"
                                                                                             ).find_next("span").text
            team_a_score = championship.find("div",{"class":"allData"}
                                      ).find_next("div", {"class":"teamsData"}
                                                 ).find_next("div", {"class":"MResult"}
                                                            ).find_next("span").text

            team_b_score = championship.find("div",{"class":"allData"}
                                      ).find_next("div", {"class":"teamsData"}
                                                 ).find_next("div", {"class":"MResult"}
                                                            ).find_next("span"
                                                                       ).find_next("span"
                                                                                  ).find_next("span").text

            match_status = championship.find("div",{"class":"matchStatus"}).find_next('span').text

            # Used getattr func to allocate None if there is no value in the webpage
            chanel_streaming = getattr(championship.find("div",{"class":"channel icon-channel"}), 'text', None)

            match_detail_link = website_URL + championship.find("div",{"class":"leftCol"}
                                                               ).find_next('a')['href']

            match_week_number = championship.find("div",{"class":"date"}).text.strip()


            # Storing the values in dictionary
            match = {'Date':date_object, 'Championship_Title':championship_title, 'Team_A':team_a, 'Team_B':team_b,
                     'Time':time, 'Team_A_Score':team_a_score, 'Team_B_Score':team_b_score,
                     'Match_Status':match_status, 'Chanel_Streaming':chanel_streaming,
                     'Match_Detail_Link':match_detail_link,'Match_Week_Number':match_week_number}

            # Appending the match to match_list
            match_details.append(match)
        
        return(match_details)
        

            

 #### Storing the data in csv file

In [5]:
# Function to store the data scraped in csv file

def store_in_csv(match_details, file_name, path):
    #C:\\Users\\yusuf\\Data Science\\Projects\\Web Scraping\\Yallakor.com"+"\\


    # Creating Keys to use in match dictionary
    keys = ['Date', 'Championship_Title', 'Team_A', 'Team_B', 'Time', 'Team_A_Score', 'Team_B_Score',
            'Match_Status', 'Chanel_Streaming', 'Match_Detail_Link','Match_Week_Number']

    # Storing the championships details in csv file
    with open(path, 'w', newline="") as match_data:
        dict_writer = csv.DictWriter(match_data, fieldnames=keys) 
        dict_writer.writeheader()

        # Looping through championships to store its data
        for match in match_details: 
            dict_writer.writerow(match)

        # Checking the file creation is done
        print(f"Match data saved to file{file_name}")

#### Calling the main function that scrape the website contect

In [9]:
# Function to do the steps of scraping
def scraping(page):
    
    src = page.content
    soup =  BeautifulSoup(src, "lxml")
    
    # List to store matches # Creating list to store match details on it
    match_details = []
    
    championships = soup.find_all("div", {'class':'matchCard'})
    
    # Test championships 
    #print(championships)
    
    
    # Calling the function that gets match info    
    match_details_scraped = get_match_info(championships)
    
    # Calling the function that stores the data scraped in csv file
    store_in_csv(match_details_scraped, file_name, path)


In [36]:
scraping(page)

Match data saved to fileMatch_Data_Scraped_03-03-2023.csv


## Working on the resulted data

### Openning the csv file and show the data

In [24]:
#Importing Libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from six.moves import urllib
import os 

In [28]:
#function to read the data

def load_data(file_path, file_name):
    
    csv_path = os.path.join(file_path, file_name)
    
    return(pd.read_csv(csv_path))



In [37]:
# Loading the data using the load_data function
data = load_data(file_path, file_name)

In [38]:
data

,Date,Championship_Title,Team_A,Team_B,Time,Team_A_Score,Team_B_Score,Match_Status,Chanel_Streaming,Match_Detail_Link,Match_Week_Number
0,2023-03-03,دوري أبطال إفريقيا,الوداد البيضاوي,فيتا كلوب,21:00,1,0,انتهت,بى ان سبورت 4 HD,https://www.yallakora.com/african-champions-le...,الجولة الاولي
1,2023-03-03,الدوري المصري,فيوتشر اف سي,الإسماعيلي,17:00,0,0,انتهت,Ontime sports 2,https://www.yallakora.com/egyptian-league/2795...,الأسبوع العشرون
2,2023-03-03,الدوري الإسباني,ريال سوسيداد,قادش,22:00,0,0,انتهت,بى ان سبورت 1HD,https://www.yallakora.com/la-liga/2764/match/8...,الأسبوع الرابع والعشرون
3,2023-03-03,بطولة أفريقيا للشباب,جامبيا - شباب,جنوب السودان - شباب,16:00,5,0,انتهت,بى ان سبورت 4 HD,https://www.yallakora.com/african-youth-champi...,دور الثمانية
4,2023-03-03,الدوري الإيطالي,نابولي,لاتسيو,21:45,0,1,انتهت,Starz Play App,https://www.yallakora.com/serie-a/2765/match/8...,الأسبوع الخامس والعشرون
5,2023-03-03,الدوري الألماني,بروسيا دورتموند,لايبزيج,21:30,2,1,انتهت,بى ان سبورت 3 HD,https://www.yallakora.com/bundesliga/2769/matc...,الأسبوع الثالث والعشرون
6,2023-03-03,الدوري الفرنسي,نيس,اوكسير,22:00,1,1,انتهت,بى ان سبورت 2 HD,https://www.yallakora.com/ligue1/2768/match/87...,الأسبوع السادس والعشرون
7,2023-03-03,الدوري التركي,ألانيا سبور,إسطنبول باشاكشهير,19:00,1,0,انتهت,NaN,https://www.yallakora.com/turkish-super-lig/27...,الأسبوع الرابع والعشرون
8,2023-03-03,الدوري السعودي,الخليج,الاتحاد,15:00,0,3,انتهت,NaN,https://www.yallakora.com/ksa-league/2784/matc...,الأسبوع التاسع عشر
9,2023-03-03,الدوري البرتغالي,بوافيستا,أروكا,21:00,0,0,انتهت,NaN,https://www.yallakora.com/primeira-liga/2775/m...,الأسبوع الثالث والعشرون
